In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')
import requests

In [ ]:
from datetime import datetime, timedelta

In [ ]:
req_reddit = requests.get('https://www.reddit.com/r/wallstreetbets/')
page_reddit = req_reddit.text

In [ ]:
# to use praw you need to get your client ID and secret from reddit, it is in preferences them app
reddit = praw.Reddit(
    client_id="YourID",
    client_secret="YourSecret",
    username="username",
    user_agent = 'Give a name'
)

In [ ]:
subreddit = reddit.subreddit('wallstreetbets')

In [ ]:
# In this part I get 1000 posts from the top rated subreddit wallstreet from last year and record post_id,
#title, score, upvote, upload time, first, last and middle time it was commented on
# in df_comment I get the comments of each post and the time they were posted
scores = []
sub_ids = [] # this is the same as submission id but out of the comment loop
submission_ids = []
upvotes = []
sub_text = []
time_uploads = []
titles = []
urls = []
time_last_visited = []
text = []
first_time_commented = []
last_time_commented = []
middle_comment_time = []
each_comment_time = []


flairs = []

parent_ids = []
comment_ids = []
comment_bodies = []
dict = {}

# Get the top most popular posts of the last year for 1000 posts
top_posts = subreddit.top(time_filter = 'year', limit = 1000)
i = 0
for submission in top_posts:
    if i % 10 == 0:
        out = i * 1. / 1000 * 100
        sys.stdout.write("\r%d%%" % out)
        sys.stdout.flush()
    
    if not submission.stickied:
        sub_ids.append(submission)
        sub_text.append(submission.selftext)
        flairs.append(submission.link_flair_text)
        titles.append(submission.title)
        scores.append(submission.score)
        time_uploads.append(datetime.fromtimestamp(submission.created))
        urls.append(submission.url)
        upvotes.append(submission.upvote_ratio)
        
        # Here I make a separate dataframe for the comments
        
        comments = submission.comments
        submission.comments.replace_more(limit = 0)
        comment_times = []
        text = []
        
        for comment in submission.comments.list():
            
            submission_ids.append(submission)
            parent_ids.append(comment.parent())
            comment_ids.append(comment.id)
            
            comment_bodies.append(comment.body)
           
            comment_times.append(comment.created_utc)
            each_comment_time.append(datetime.fromtimestamp(comment.created_utc))
        text= comment_bodies   
        #This will make a dictionary of the comments    
        dict[str(submission)] = text    
        
        sorted_comments_times = sorted(comment_times)
        
        # Here I will get the time for the middle comment on a post
        middle_comment = len(sorted_comments_times)/2
      
        middle_comment_time.append(datetime.fromtimestamp(comment_times[int(middle_comment)]))
        
        
        first_time_commented.append(datetime.fromtimestamp(sorted_comments_times[0]))
        last_time_commented.append(datetime.fromtimestamp(sorted_comments_times[-1]))
        i = i + 1

        
df_comments = pd.DataFrame(list(zip(submission_ids, parent_ids, comment_ids,each_comment_time, comment_bodies)),
                           columns = ['submission_id','parent_id','comment_id','comment_time','comment'])    

df_submission = pd.DataFrame(list(zip(sub_ids,flairs, titles, scores, upvotes, time_uploads,first_time_commented,last_time_commented,middle_comment_time, urls, sub_text)), 
                             columns = ['submission_id','Flair','title','score','upvotes','time_upload','first_time_commented','last_time_commented','middle_comment_time','url', 'text'])
     

df_comments.to_csv('filename.csv')            
df_submission.to_csv('filename.csv')   

In [ ]:
df_submission['duration_middle'] = pd.to_datetime(df_submission.middle_comment_time) - pd.to_datetime(df_submission.first_time_commented)
df_submission['duration_total'] = pd.to_datetime(df_submission.last_time_commented)- pd.to_datetime(df_submission.first_time_commented)

In [ ]:
# this part visualized the time evolution of the comments on least and most popular posts
low_scores = df_submission.loc[df_submission.score < 26500, 'submission_id']
high_scores = df_submission.loc[df_submission.score > 200000, 'submission_id']
ids = low_scores.append(high_scores)
ids.reset_index(drop = True, inplace = True)
binwidth = 1
fig,ax = plt.subplots(int(len(ids)/2),2,figsize=(12,20))
for i in range(len(ids)):
    submission = ids[i]
    scores = df_submission.loc[df_submission.submission_id == submission, 'score']
    times = df_comments.loc[df_comments.submission_id == submission, 'comment_time']
    sorted_times = sorted(times)
    durations = pd.to_datetime(times) - pd.to_datetime(sorted_times[0])
    comment_time_from_start = [t.total_seconds()/3600 for t in durations]
    ax[i//2,i%2].hist(comment_time_from_start,
                      bins=np.arange(min(comment_time_from_start), max(comment_time_from_start) + binwidth, binwidth),
                      color = 'b', label = ('subm = '+ str(submission)+'  And  '+'scores = '+ scores.to_string(index = False)))
    ax[i//2,i%2].set_xlim(left = 0, right = 50)
    ax[i//2,i%2].set_xlabel('Comment_time', fontsize = 14)
    ax[i//2, i%2].set_ylabel('Frequency', fontsize = 14)
    ax[i//2, i%2].legend(loc = 'best', fontsize = 10)
#plt.savefig('d:\\Machine_Learning\\Erdoes_Fall\\project\\comment_time_hist_lowHigh_upvotes.jpeg')
#plt.savefig('d:\\Machine_Learning\\Erdoes_Fall\\project\\comment_time_hist_lowHigh_upvotes.svg')
plt.show()